# Experiment 2 - Obtain the result of optimization at each scale 

- Compare the downsampled versions of the images and the reconstructions.

- 7-10 initial guesses

-  Use 0.5 overlap


In [1]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwc.multires.class_multiressolver import *
import matplotlib.pyplot as plt
import torch
from src.cpwc.tools.ptychography import Ptychography as Ptychography
from src.cpwc.tools.utils import *
torch.cuda.empty_cache()
from src.utils.cuda_pyth import empty_cuda, print_cuda_tensors
from src.utils.get_image import get_image
from src.utils.noise import add_gaussian_noise, add_poisson_noise
from src.utils.metrics import get_ring_average, F2fluxconverter
from src.utils.manage_data import save_data,unwrap_2d,extract_data
# Set seeds 

device = "cuda" if torch.cuda.is_available() else "cpu"
max_scale = 9
image,image_tensor_ = get_image("samples/potential.npy", max_scale,device)
max_probe_size = 128
cycle = [0, -1, -1, -1,  1, 1, 1]
tol = [1e-10] * 9
tol_in = [1e-15] * 9
max_shift = 64
lmbda = 1e-30
#noise = 1e30
linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)
multires = MultiRes(max_scale, device)
image_tensor__ = linOperator.apply(image_tensor_)
#image_tensor__ = add_poisson_noise(image_tensor__, noise, 'cuda')
loss = Loss(linOperator,image_tensor__,lmbda= lmbda)

/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1 - Downsampled version of the original image 

In [2]:
image_8 = image[::8,::8]
image_4 = image[::4,::4]
image_2 = image[::2,::2]
image_1 = image

np.save("experiments/results/exp2/gt_image_8.npy",image_8)
np.save("experiments/results/exp2/gt_image_4.npy",image_4)
np.save("experiments/results/exp2/gt_image_2.npy",image_2)
np.save("experiments/results/exp2/gt_image_1.npy",image_1)

In [ ]:
plt.figure(figsize = (15,5))
plt.subplot(1,4,1)
plt.imshow(image_8)
plt.title("Downsampled by 8")
plt.subplot(1,4,2)
plt.imshow(image_4)
plt.title("Downsampled by 4")
plt.subplot(1,4,3)
plt.imshow(image_2)
plt.title("Downsampled by 2")
plt.subplot(1,4,4)
plt.imshow(image_1)
plt.title("Original GT")
plt.show()
plt.tight_layout()
plt.savefig("experiments/results/exp2/gt_images.png")


In [4]:
I_in = 25*np.array([1, 15, 10, 5, 10, 30, 100])
#I_out = 10*np.array([0, 0, 0, 30, 10,10,300])
I_out = 10*np.array([0, 0, 0, 10, 5,5,300])
#I_out = 10*np.array([0, 0, 0, 0, 0,0,350])

In [ ]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
LR = 1e-2
early_stopping = 9
model = MultiResSolver(multires, 
                        loss, 
                        LR = LR,
                        I_in = I_in,
                        I_out = I_out,
                        tol = tol,
                        tol_in = tol_in,
                        cycle = cycle,
                        l1_type = "l1_row",
                        gt = image_tensor_,
                        early_stopping = early_stopping)
model.solve_multigrid()
model.print_time()

In [ ]:
phase = torch.angle(model.c_k[0,0,:,:].to('cpu'))
mean_img = np.mean(image)
phase = phase.numpy()
phase = unwrap_2d(phase)
phase += (mean_img-np.mean(phase)) 
np.save("experiments/results/exp2/image{}_seed_{}.npy".format(early_stopping,seed),phase)

In [ ]:
image6_0 = np.load("experiments/results/exp2/image6_seed_0.npy")
image6_10 = np.load("experiments/results/exp2/image6_seed_10.npy")
image6_20 = np.load("experiments/results/exp2/image6_seed_20.npy")
image6_30 = np.load("experiments/results/exp2/image6_seed_30.npy")
image6_40 = np.load("experiments/results/exp2/image6_seed_40.npy")
image6_50 = np.load("experiments/results/exp2/image6_seed_50.npy")
image6_60 = np.load("experiments/results/exp2/image6_seed_60.npy")

image7_0 = np.load("experiments/results/exp2/image7_seed_0.npy")
image7_10 = np.load("experiments/results/exp2/image7_seed_10.npy")
image7_20 = np.load("experiments/results/exp2/image7_seed_20.npy")
image7_30 = np.load("experiments/results/exp2/image7_seed_30.npy")
image7_40 = np.load("experiments/results/exp2/image7_seed_40.npy")
image7_50 = np.load("experiments/results/exp2/image7_seed_50.npy")
image7_60 = np.load("experiments/results/exp2/image7_seed_60.npy")

image8_0 = np.load("experiments/results/exp2/image8_seed_0.npy")
image8_10 = np.load("experiments/results/exp2/image8_seed_10.npy")
image8_20 = np.load("experiments/results/exp2/image8_seed_20.npy")
image8_30 = np.load("experiments/results/exp2/image8_seed_30.npy")
image8_40 = np.load("experiments/results/exp2/image8_seed_40.npy")
image8_50 = np.load("experiments/results/exp2/image8_seed_50.npy")
image8_60 = np.load("experiments/results/exp2/image8_seed_60.npy")




In [ ]:
plt.figure(figsize=(10,30))
plt.subplot(8,3,1)
plt.imshow(image_8)
plt.title("GT Downsampled by 8")
plt.subplot(8,3,2)
plt.imshow(image_4)
plt.title("GT Downsampled by 4")
plt.subplot(8,3,3)
plt.imshow(image_2)
plt.title("GT Downsampled by 2")
plt.subplot(8,3,4)
plt.imshow(image6_0)
plt.title("Seed 0")
plt.subplot(8,3,5)
plt.imshow(image7_0)
plt.title("Seed 0")
plt.subplot(8,3,6)
plt.imshow(image8_0)
plt.title("Seed 0")
plt.subplot(8,3,7)
plt.imshow(image6_10)
plt.title("Seed 10")
plt.subplot(8,3,8)
plt.imshow(image7_10)
plt.title("Seed 10")
plt.subplot(8,3,9)
plt.imshow(image8_10)
plt.title("Seed 10")
plt.subplot(8,3,10)
plt.imshow(image6_20)
plt.title("Seed 20")
plt.subplot(8,3,11)
plt.imshow(image7_20)
plt.title("Seed 20")
plt.subplot(8,3,12)
plt.imshow(image8_20)
plt.title("Seed 20")
plt.subplot(8,3,13)
plt.imshow(image6_30)
plt.title("Seed 30")
plt.subplot(8,3,14)
plt.imshow(image7_30)
plt.title("Seed 30")
plt.subplot(8,3,15)
plt.imshow(image8_30)
plt.title("Seed 30")
plt.subplot(8,3,16)
plt.imshow(image6_40)
plt.title("Seed 40")
plt.subplot(8,3,17)
plt.imshow(image7_40)
plt.title("Seed 40")
plt.subplot(8,3,18)
plt.imshow(image8_40)
plt.title("Seed 40")
plt.subplot(8,3,19)
plt.imshow(image6_50)
plt.title("Seed 50")
plt.subplot(8,3,20)
plt.imshow(image7_50)
plt.title("Seed 50")
plt.subplot(8,3,21)
plt.imshow(image8_50)
plt.title("Seed 50")
plt.subplot(8,3,22)
plt.imshow(image6_60)
plt.title("Seed 60")
plt.subplot(8,3,23)
plt.imshow(image7_60)
plt.title("Seed 60")
plt.subplot(8,3,24)
plt.imshow(image8_60)
plt.title("Seed 60")
plt.tight_layout()
plt.savefig("experiments/results/exp2/seed_comparison.png")




In [ ]:
iter = 10
I_out = 10*np.array([0, 0, 0, 0, 0,0,iter])
seed = 40
torch.manual_seed(seed)
np.random.seed(seed)
LR = 1e-1
early_stopping = 9
model = MultiResSolver(multires, 
                        loss, 
                        LR = LR,
                        I_in = I_in,
                        I_out = I_out,
                        tol = tol,
                        tol_in = tol_in,
                        cycle = cycle,
                        l1_type = "l1_row",
                        gt = image_tensor_,
                        early_stopping = early_stopping)
model.solve_multigrid()
model.print_time()

----------- s = 9 -----------
----------- s = 8 -----------
----------- s = 7 -----------
----------- s = 6 -----------
----------- s = 7 -----------
----------- s = 8 -----------
----------- s = 9 -----------
1
Iter 1, [loss,mse,reg,csim] : [129112.8426842, 129112.8426842, 0.0, 0.0300197, ] 
LR = 0.8
1
Iter 2, [loss,mse,reg,csim] : [60675.2328638, 60675.2328638, 0.0, 0.2818802, ] 
LR = 0.8
1
Iter 3, [loss,mse,reg,csim] : [44568.8944736, 44568.8944736, 0.0, 0.3443303, ] 
LR = 0.8
1
Iter 4, [loss,mse,reg,csim] : [37357.02276, 37357.02276, 0.0, 0.3875906, ] 
LR = 0.8
1
Iter 5, [loss,mse,reg,csim] : [32290.6669575, 32290.6669575, 0.0, 0.4195629, ] 
LR = 0.8
1
Iter 6, [loss,mse,reg,csim] : [28713.3110949, 28713.3110949, 0.0, 0.4397185, ] 
LR = 0.8
1
Iter 7, [loss,mse,reg,csim] : [25572.2118978, 25572.2118978, 0.0, 0.4608697, ] 
LR = 0.8
1
Iter 8, [loss,mse,reg,csim] : [22264.280612, 22264.280612, 0.0, 0.484175, ] 
LR = 0.8
1
Iter 9, [loss,mse,reg,csim] : [19414.6359654, 19414.6359654, 0.0,

In [14]:
phase = torch.angle(model.c_k[0,0,:,:].to('cpu'))
mean_img = np.mean(image)
phase = phase.numpy()
phase = unwrap_2d(phase)
phase += (mean_img-np.mean(phase)) 
np.save("experiments/results/exp2/gd_image{}_seed_{}.npy".format(iter,seed),phase)